In [277]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd
from datetime import date, datetime
import holidays
import requests
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all"
import ast
pd.options.display.max_columns = None

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Barra de progreso de un proceso
# ------------------------------------------------------------------------------
from tqdm import tqdm

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

# Poner descansos en código.
# ------------------------------------------------------------------------------
import time

# Librería para crear archivos pickle.
# ------------------------------------------------------------------------------
import pickle
import os

## Predicción empleando una API de previsión meteorológica

Coordenadas Washington D.C., USA:

38.897588, -77.036496

In [278]:
#Creamos un diccionario con la localización de Washington D.C., con la posibilidad de añadir más localizaciones.
dict_lugares={'Washington':[38.897588, -77.036496]}

In [279]:
def obtener_clima(dicc, producto):
    #Iteramos por el diccionario con las localizaciones
    lista_dataframes = []
    for key, value in dicc.items():
        lat = value[0]
        lon = value[1]
    #Llamamos a la API
        url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
        response = requests.get(url=url)
        codigo_estado = response.status_code
        razon_estado = response.reason
    #Ponemos esto para comprobar si todo está correcto.    
        if codigo_estado == 200:
            print('La peticion de se ha realizado correctamente, código de estado:',codigo_estado,'razón:',razon_estado)
        elif codigo_estado == 402:
            print('No se ha podido autorizar al usuario, código de estado:', codigo_estado,'razón:',razon_estado)
        elif codigo_estado == 404:
            print('Algo ha salido mal, el recurso no se ha encontrado,código de estado:', codigo_estado,'razón:',razon_estado)
        else:
            print('Ha ocurrido algo inesperado, código de estado:', codigo_estado,'razón:',razon_estado)
    #Convertimos los resultados en un dataframe:
        df = pd.DataFrame.from_dict(pd.json_normalize(response.json()['dataseries']))
    #Creamos columnas con el nombre del pais, la latitud y la longitud para poder hacer la union.
        df["place"] = key
        df["latitud"] = lat
        df["longitud"] = lon
    #Apendeamos el dataframe en una lista que hemos creado
        lista_dataframes.append(df)
    
    #Unimos los dataframes en uno
    df_lugares = pd.concat(lista_dataframes, axis= 0, ignore_index= True)
    return df_lugares

In [280]:
#Llamamos a la función para obtener la predicción de los próximos 7 días.
df_meteo = obtener_clima(dict_lugares,'civillight')
df_meteo

La peticion de se ha realizado correctamente, código de estado: 200 razón: OK


,date,weather,wind10m_max,temp2m.max,temp2m.min,place,latitud,longitud
0,20230212,pcloudy,2,11,10,Washington,38.897588,-77.036496
1,20230213,clear,3,11,9,Washington,38.897588,-77.036496
2,20230214,cloudy,3,12,11,Washington,38.897588,-77.036496
3,20230215,cloudy,3,12,12,Washington,38.897588,-77.036496
4,20230216,mcloudy,3,12,11,Washington,38.897588,-77.036496
5,20230217,pcloudy,3,12,11,Washington,38.897588,-77.036496
6,20230218,cloudy,3,12,11,Washington,38.897588,-77.036496


In [281]:
df_prueba1=df_meteo.copy()
df_prueba1.columns

Index(['date', 'weather', 'wind10m_max', 'temp2m.max', 'temp2m.min', 'place',
       'latitud', 'longitud'],
      dtype='object')

In [282]:
#Obtenemos la fecha de la que estamos haciendo la predicción.
df_prueba1['fecha_nueva'] = pd.to_datetime(df_prueba1["date"], format="%Y%m%d")

In [283]:
#Obtenemos si ese día es festivo o no.
df_prueba1['festividad'] = pd.Series(df_prueba1.fecha_nueva).apply(lambda x: holidays.CountryHoliday('US').get(x)).values
df_prueba1['festividad_1'] = df_prueba1['festividad'].astype('bool').astype('int')

/home/solkiria/anaconda3/lib/python3.9/site-packages/holidays/utils.py:266: DeprecationWarning: CountryHoliday is deprecated, use country_holidays instead.
  warnings.warn(


In [284]:
#Obtenemos la columna del año
df_prueba1['año_bueno'] = pd.DatetimeIndex(df_prueba1['fecha_nueva']).year

In [285]:
#Creamos una función para obtener la estación a la que pertenece esa fecha.
def sacar_estacion(fecha):
    if fecha.month == 2 or fecha.month == 1:
        return "invierno"
    elif fecha.month == 3 and fecha.day <= 20:
        return "invierno"
    elif fecha.month == 12 and fecha.day >20:
        return "invierno"
    elif fecha.month == 4 or fecha.month == 5:
        return "primavera"
    elif fecha.month == 3 and fecha.day >20:
        return "primavera"
    elif fecha.month == 6 and fecha.day <=20:
        return "primavera"
    elif fecha.month == 7 or fecha.month == 8:
        return "verano"
    elif fecha.month == 6 and fecha.day > 20:
        return "verano"
    elif fecha.month == 9 and fecha.day < 23:
        return "verano"
    elif fecha.month == 10 or fecha.month == 11:
        return "otoño"
    elif fecha.month == 9 and fecha.day > 22:
        return "otoño"
    elif fecha.month == 12 and fecha.day <= 20:
        return "otoño"

In [286]:
#Obtenemos la estación
df_prueba1["estacion_correcta"] = df_prueba1["fecha_nueva"].apply(sacar_estacion)

In [287]:
#Obtenemos el mes
df_prueba1['mes_bueno'] = pd.DatetimeIndex(df_prueba1['fecha_nueva']).month

In [288]:
#Obtenemos el día de la semana
df_prueba1['dia_semana_nuevo']=df_prueba1['fecha_nueva'].dt.weekday

In [289]:
#Creamos una funcion para crear una columna con los dias laborales.
def sacar_laboral(df):
    if (df['dia_semana_nuevo']<=4) and (df['festividad_1']== 0):
        return 1
    else:
        return 0

In [290]:
#Obtenemos los días laborales
df_prueba1['no_laboral_nuevo']=df_prueba1.apply(sacar_laboral,axis=1)

Tiempos meterológicos del modelo:
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

Posibles tiempos meteorologicos de la API clasificados según nuestro modelo:
- 1: clear,pcloudy,mcloudy,cloudy
- 2: humid
- 3: lightrain, oshower, ishower, lightsnow,ts
- 4: rain, rainsnow, tsrain, snow

In [291]:
#Obtenemos el clima:
condiciones=[df_prueba1['weather'].isin(['clear','pcloudy','mcloudy','cloudy']),
            df_prueba1['weather'].isin(['humid']),
            df_prueba1['weather'].isin(['lightrain', 'oshower', 'ishower', 'lightsnow','ts']),
            df_prueba1['weather'].isin(['rain', 'rainsnow', 'tsrain', 'snow'])]
opciones=[1,2,3,4]
df_prueba1['clima'] =np.select(condiciones,opciones)

In [292]:
#Obtenemos la temperatura
def obtener_temp(df):
    df['temperatura']=(df['temp2m.min'] + df['temp2m.max'])/2

In [293]:
obtener_temp(df_prueba1)

In [294]:
#Obtenemos viento
df_prueba1['viento_ms']=df_prueba1['wind10m_max']

In [295]:
def descodificar_viento(df):
    viento_map={1:0,
            2:(((0.3+3.4)/2)*3600)/1000,
            3:(((3.4+8.0)/2)*3600)/1000,
            4:(((8.0+10.8)/2)*3600)/1000,
            5:(((10.8+17.2)/2)*3600)/1000,
            6:(((17.2+24.5)/2)*3600)/1000,
            7:(((24.5+32.6)/2)*3600)/1000,
            8:((33)*3600)/1000}
    df['viento']=df['viento_ms'].map(viento_map)
    df.drop(['viento_ms'],axis=1,inplace=True)
    return df

In [296]:
df_prueba1=descodificar_viento(df_prueba1)

In [297]:
#Obtenemos humedad
df_prueba1['humedad']=65

In [298]:
#Obtenemos la sensacion termica
df_prueba1['sens_termica'] = df_prueba1['temperatura']

In [299]:
df_prueba1

,date,weather,wind10m_max,temp2m.max,temp2m.min,place,latitud,longitud,fecha_nueva,festividad,festividad_1,año_bueno,estacion_correcta,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,clima,temperatura,viento,humedad,sens_termica
0,20230212,pcloudy,2,11,10,Washington,38.897588,-77.036496,2023-02-12,None,0,2023,invierno,2,6,0,1,10.5,6.66,65,10.5
1,20230213,clear,3,11,9,Washington,38.897588,-77.036496,2023-02-13,None,0,2023,invierno,2,0,1,1,10.0,20.52,65,10.0
2,20230214,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-14,None,0,2023,invierno,2,1,1,1,11.5,20.52,65,11.5
3,20230215,cloudy,3,12,12,Washington,38.897588,-77.036496,2023-02-15,None,0,2023,invierno,2,2,1,1,12.0,20.52,65,12.0
4,20230216,mcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-16,None,0,2023,invierno,2,3,1,1,11.5,20.52,65,11.5
5,20230217,pcloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-17,None,0,2023,invierno,2,4,1,1,11.5,20.52,65,11.5
6,20230218,cloudy,3,12,11,Washington,38.897588,-77.036496,2023-02-18,None,0,2023,invierno,2,5,0,1,11.5,20.52,65,11.5


In [300]:
df_prueba1.columns

Index(['date', 'weather', 'wind10m_max', 'temp2m.max', 'temp2m.min', 'place',
       'latitud', 'longitud', 'fecha_nueva', 'festividad', 'festividad_1',
       'año_bueno', 'estacion_correcta', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'clima', 'temperatura', 'viento', 'humedad',
       'sens_termica'],
      dtype='object')

In [301]:
nuevo_orden=['clima','temperatura','humedad', 'viento', 'festividad_1','mes_bueno','dia_semana_nuevo',
                'no_laboral_nuevo','estacion_correcta','año_bueno',
                'date', 'weather', 'wind10m_max', 'temp2m.max', 'temp2m.min', 'place',
                'latitud', 'longitud', 'fecha_nueva', 'festividad',
                'sens_termica']
df_prueba2=df_prueba1.reindex(nuevo_orden,axis=1)

In [302]:
df_prueba2.drop(['date', 'weather', 'wind10m_max', 'temp2m.max', 'temp2m.min', 'place',
                'latitud', 'longitud', 'fecha_nueva', 'festividad',
                'sens_termica'], axis=1,inplace=True)
df_prueba2.head(3)

,clima,temperatura,humedad,viento,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_correcta,año_bueno
0,1,10.5,65,6.66,0,2,6,0,invierno,2023
1,1,10.0,65,20.52,0,2,0,1,invierno,2023
2,1,11.5,65,20.52,0,2,1,1,invierno,2023


In [303]:
df_prueba3=df_prueba2.copy()

In [304]:
def codificar_año23(df):
    condicion=df['año_bueno'].isin([2023])
    verdad=1
    falso=0
    df['año_map_18']=0
    df['año_map_19']=0
    df['año_map_23']=np.where(condicion,verdad,falso)
    df.drop(['año_bueno'],axis=1,inplace=True)
    return df

In [305]:
df_prueba3=codificar_año23(df_prueba3)

In [306]:
def codificar_estacion(df):
    estacion_map={'invierno':0,'primavera':1,'otoño':2,'verano':3}
    df['estacion_map']=df['estacion_correcta'].map(estacion_map)
    df.drop(['estacion_correcta'],axis=1,inplace=True)
    return df

In [307]:
df_prueba3=codificar_estacion(df_prueba3)

In [308]:
df_prueba3.head(3)

,clima,temperatura,humedad,viento,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map_18,año_map_19,año_map_23,estacion_map
0,1,10.5,65,6.66,0,2,6,0,0,0,1,0
1,1,10.0,65,20.52,0,2,0,1,0,0,1,0
2,1,11.5,65,20.52,0,2,1,1,0,0,1,0


In [309]:
df_prueba3=df_prueba3.reindex(['clima', 'temperatura', 'humedad', 'viento', 'festividad_1',
       'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo', 'estacion_map',
       'año_map_23', 'año_map_18', 'año_map_19'],axis=1)
df_prueba3

,clima,temperatura,humedad,viento,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_map,año_map_23,año_map_18,año_map_19
0,1,10.5,65,6.66,0,2,6,0,0,1,0,0
1,1,10.0,65,20.52,0,2,0,1,0,1,0,0
2,1,11.5,65,20.52,0,2,1,1,0,1,0,0
3,1,12.0,65,20.52,0,2,2,1,0,1,0,0
4,1,11.5,65,20.52,0,2,3,1,0,1,0,0
5,1,11.5,65,20.52,0,2,4,1,0,1,0,0
6,1,11.5,65,20.52,0,2,5,0,0,1,0,0


## Ahora ya podemos hacer la predicción!

In [310]:
#Abrimos el pickle creado y aplicamos el modelo almacenado en él.
with open('data/modelo_rf_totales.pkl', 'rb') as f:
    forest=pickle.load(f)

In [311]:
#Comprobamos que el modelo se ha cargado adecuadamente.
forest

RandomForestRegressor(max_depth=15, max_features=3, min_samples_leaf=2,
                      min_samples_split=1)

In [312]:
forest.predict(df_prueba3)

array([2459.84470455, 2008.03068708, 2136.16747951, 2252.97378109,
       2351.86510834, 2392.00360834, 2403.29318683])

In [313]:
df_prueba4=df_prueba3.copy()

In [314]:
df_prueba4['bicis_previstas']=forest.predict(df_prueba3)
df_prueba4['bicis_previstas']=df_prueba4['bicis_previstas'].round(0).astype(int)

In [315]:
df_prediccion_bicis=df_prueba4.copy()
df_prediccion_bicis=df_prediccion_bicis.reindex(['bicis_previstas','clima', 'temperatura', 'humedad', 'viento', 'festividad_1',
       'mes_bueno', 'dia_semana_nuevo', 'no_laboral_nuevo', 'estacion_map',
       'año_map_23', 'año_map_18', 'año_map_19'],axis=1)
df_prediccion_bicis=pd.concat([df_prueba1['fecha_nueva'],df_prediccion_bicis],axis=1)
df_prediccion_bicis

,fecha_nueva,bicis_previstas,clima,temperatura,humedad,viento,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,estacion_map,año_map_23,año_map_18,año_map_19
0,2023-02-12,2460,1,10.5,65,6.66,0,2,6,0,0,1,0,0
1,2023-02-13,2008,1,10.0,65,20.52,0,2,0,1,0,1,0,0
2,2023-02-14,2136,1,11.5,65,20.52,0,2,1,1,0,1,0,0
3,2023-02-15,2253,1,12.0,65,20.52,0,2,2,1,0,1,0,0
4,2023-02-16,2352,1,11.5,65,20.52,0,2,3,1,0,1,0,0
5,2023-02-17,2392,1,11.5,65,20.52,0,2,4,1,0,1,0,0
6,2023-02-18,2403,1,11.5,65,20.52,0,2,5,0,0,1,0,0
